In [1]:
pd.__version__

NameError: name 'pd' is not defined

In [2]:
import collections
from datetime import date, timedelta
import os
from socket import timeout
import sys

import ee
from ee import ImageCollection
import geemap
import numpy as np
import pandas as pd
ee.Initialize()

# Test if shapefile works

In [18]:
!pwd

/home/hassib12/code/hassib12/FoodSecurityForecaster/notebooks


In [56]:
states_shp = f'../raw_data/IND/admin1/ind.shp'
ee_shape = geemap.shp_to_ee(states_shp)

In [57]:
county_Bahr_al_Jabal = ee_shape.filter(ee.Filter.eq("NAME1_", 'Assam'))

In [58]:
type(county_Bahr_al_Jabal)

ee.featurecollection.FeatureCollection

In [59]:
map = geemap.Map()
map.addLayer(county_Bahr_al_Jabal)
map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [ ]:
state_names = pd.read_csv('../raw_data/state_names_USA.csv')['NAME1_,C,33']
state_names.head(2)

In [ ]:
def download_map(country_code,
                 state_name,
                 date_range=('2021-01-01','2022-12-30'),
                 modis_collection='006/MOD13A1',
                 num_pixels=1000):
    Map = geemap.Map()
    lower_case_country_code = country_code.lower
    states_shp = f'../raw_data/USA/admin1/USA.shp'
 
    ee_shape = geemap.shp_to_ee(states_shp)
    state_ee = ee_shape.filter(ee.Filter.eq("NAME1_,C,33", state_name))

    
    Map.addLayer(state_ee.geometry())

    image_collection = ee.ImageCollection(f'MODIS/{modis_collection}').filter(ee.Filter.date(*date_range)).toBands()

    region = state_ee.geometry()

    samples = image_collection.sample(region, scale=300 , numPixels=num_pixels)

    print("samples:", samples)
    features = samples.getInfo()['features']
    print("features:", features)

    data = []
    for feature_set in features:
        
        for k, v in feature_set['properties'].items():
            date, property_name = k[:10], k[11:]
            date = date.replace('_', '-')
            data.append({'band': property_name, 'date': date, 'value': v})

    df = pd.DataFrame(data)
    df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
    df.attrs['state_name'] = state_name
    df['band'] = df['band'].astype("category")

    return df

def load_all(country_code='SSD', admin_level='admin1', date_range=('2010-01-01', '2018-01-01'), modis_collection='006/MOD13A1', num_pixels=1000):
    lower_case_country_code = country_code.lower
    states_shp = f'../raw_data/USA/admin1/USA.shp'

    #ee_shape = geemap.shp_to_ee(states_shp)
    #geemap.common.ee_to_csv(ee_shape, "tmp.csv", timeout=1000)

    # state_names = pd.read_csv('tmp.csv')['NAME1_']
    # os.remove('tmp.csv') 
    
    for state_name in state_names:
        print(f'working on:{state_name}')
        df = download_map(country_code, state_name, date_range, modis_collection, num_pixels)
        df.to_pickle(f'../raw_data/raw_pixels/USA/{state_name}.zip')
        print(f"Downloaded data for {state_name}...")

# if __name__ == "__main__":
#     ee.Initialize()
#     load_all(sys.argv[1])


In [ ]:
load_all('USA')

# SOUTH SUDAN

In [ ]:
state_names = pd.read_csv('../raw_data/state_names_SSD.csv')['NAME_2,C,13']
state_names.head()

In [ ]:
def download_map(country_code,
                 state_name,
                 date_range=('2021-01-01','2022-12-30'),
                 modis_collection='006/MOD13A1',
                 num_pixels=1000):
    Map = geemap.Map()
    lower_case_country_code = country_code.lower
    states_shp = f'../raw_data/raw_data/gadm41_SSD_shp/gadm41_SSD_2.shp'
 
    ee_shape = geemap.shp_to_ee(states_shp)
    state_ee = ee_shape.filter(ee.Filter.eq(["NAME_2"], state_name))
    Map.addLayer(state_ee.geometry())

    image_collection = ee.ImageCollection(f'MODIS/{modis_collection}').filter(ee.Filter.date(*date_range)).toBands()

    region = state_ee.geometry()

    samples = image_collection.sample(region, scale=300 , numPixels=num_pixels)

    features = samples.getInfo()['features']

    data = []
    for feature_set in features:
        for k, v in feature_set['properties'].items():
            date, property_name = k[:10], k[11:]
            date = date.replace('_', '-')
            data.append({'band': property_name, 'date': date, 'value': v})

    df = pd.DataFrame(data)
    df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
    df.attrs['state_name'] = state_name
    df['band'] = df['band'].astype("category")

    return df

def load_all(country_code='SSD', admin_level='admin1', date_range=('2010-01-01', '2018-01-01'), modis_collection='006/MOD13A1', num_pixels=1000):
    lower_case_country_code = country_code.lower
    states_shp = f'../raw_data/raw_data/gadm41_SSD_shp/gadm41_SSD_2.shp'

    #ee_shape = geemap.shp_to_ee(states_shp)
    #geemap.common.ee_to_csv(ee_shape, "tmp.csv", timeout=1000)

    # state_names = pd.read_csv('tmp.csv')['NAME1_']
    # os.remove('tmp.csv') 
    
    for state_name in state_names:
        print(f'working on:{state_name}')
        df = download_map(country_code, state_name, date_range, modis_collection, num_pixels)
        df.to_pickle(f'../raw_data/raw_pixels/{state_name}.zip')
        print(f"Downloaded data for {state_name}...")

# if __name__ == "__main__":
#     ee.Initialize()
#     load_all(sys.argv[1])

In [ ]:
load_all()